In [1]:
OUTPUT_PATH = '../../data/'

In [2]:
%run '../../schema_definition/schema.py'

In [3]:
import sys
sys.path.insert(0, '../../')

from schema_definition.schema import *

In [4]:
from pyspark.shell import spark
from pyspark.sql import functions as F

21/10/16 04:14:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/16 04:14:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/10/16 04:14:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/

Using Python version 3.8.9 (default, Aug 21 2021 15:53:23)
Spark context Web UI available at http://localhost:4042
Spark context available as 'sc' (master = local[*], app id = local-1634318074587).
SparkSession available as 'spark'.


- When the number of events reach 1000, output the events to a JSON file.
- The output filenames should have a batch number e.g. the second 1000 records will go into a file called 2.json and so on.

In [7]:
gold_df = spark.read.parquet(OUTPUT_PATH + "gold")
calculate_response = gold_df.withColumn("Response", F.datediff(F.col("RequestDate"), F.col("ImplementedDate")))
calculate_response.createOrReplaceTempView("fastest_response")

fastest_response_query = """
                        SELECT *
                        FROM fastest_response
                        ORDER BY Response DESC
                        """
fastest_response_df = spark.sql(fastest_response_query)
# fastest_response_df.coalesce(1).write.csv(OUTPUT_PATH + 'response', sep=',', header=True, mode='overwrite')
fastest_response_df.coalesce(1).write.option("maxRecordsPerFile", 1000).json(OUTPUT_PATH + 'response', mode='overwrite')
fastest_response_df.toPandas()

,AccountID,CODE,ImplementedDate,ActiveIndicator,AccountType,Service,BU,RequestDate,AccountStatus,StatusCode,Amount,Version,AgentID,Fibre,LastUpdatedDate,PropertyType,PostCode,HashKey,date,Response
0,415740,105,2017-09-05,1.0,B2B,ELE,NUOS,2017-09-04,PAID,8,3.31,v1_0,307312,EA983_54004586547-N-11,2017-09-11 10:44:00,RESI,2581,9057e3e8fc73f87b752b99e782400367c2ec2390358e72...,2021-10-16,-1
1,524218,105,2017-09-05,1.0,B2B,ELE,NUOS,2017-09-04,PAID,7,1.01,v1_0,307312,EA983_146001477792-N-3,2017-09-11 10:45:00,RESI,2581,51ef88b3efa024e275f278889898e1717cdbaba58b8c52...,2021-10-16,-1
2,632695,105,2017-09-05,1.0,B2B,ELE,NUOS,2017-09-04,PAID,3,0.09,v1_0,307312,EA983_50004634622-N-13,2017-09-11 09:27:00,RESI,2581,582e91375d8d54dbd5f60f7e32a84e1fa25c678e64ff7e...,2021-10-16,-1
3,10036,105,2017-09-05,1.0,B2B,ELE,NUOS,2017-09-04,PAID,2,1.04,v1_0,307312,EA983_26004938352-N-3,2017-09-11 10:44:00,RESI,2581,342d3532b66d6a9ed253470d95a468ebe3da9122f241cd...,2021-10-16,-1
4,10026,105,2017-09-05,1.0,B2B,ELE,NUOS,2017-09-04,PAID,3,13.12,v1_0,307312,EA983_26004938310-N-4,2017-09-11 10:44:00,RESI,2581,bf60bf55a245767b00dace0a2e467fe09587822575e3c5...,2021-10-16,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29648,10018,107,2017-09-11,1.0,B2B,ELE,NUOS,2016-09-08,PAID,8,-1.75,v1_0,307664,CEN99573813-C-8,2017-09-18 13:13:00,RESI,2582,c1b7a3cfd9da5aadbfed6845a65ead2d9f288de6893b9b...,2021-10-16,-368
29649,10047,107,2017-09-11,1.0,B2B,ELE,NUOS,2016-08-29,PAID,4,-38.49,v1_0,307664,CEN98648458-C-8,2017-09-18 13:13:00,RESI,2582,34e79972cc501cbdbe434cbb28bce4b3d0cd6b57729653...,2021-10-16,-378
29650,10039,107,2017-09-11,1.0,B2B,ELE,NUOS,2016-07-14,PAID,0,-0.59,v1_0,307662,CEN94977071-C-2,2017-09-18 13:14:00,RESI,2582,260b832faef041ba358c15077b3db8b7ed10855526b1cd...,2021-10-16,-424
29651,10037,107,2017-09-11,1.0,B2B,ELE,NUOS,2014-11-26,PAID,7,-10.46,v1_0,307662,CEN51544029-C-2,2017-09-18 13:14:00,RESI,2582,c43daf1377ee14c9b40c34a25eff9b49c03e613f830d33...,2021-10-16,-1020
